# Constantes

In [1]:
CHECKSUM_PATH = "./checksum-xmls.csv"
XSL = "../../helpers/reader/passage.transform.xsl"

# Récupération des éléments du corpus

In [2]:
import glob
from MyCapytain.resolvers.cts.local import CtsCapitainsLocalResolver

repositories = list(glob.glob("../../data/raw/corpora/**/*", recursive=False))
resolver = CtsCapitainsLocalResolver(repositories)

../../data/raw/corpora/lascivaroma_additional-texts/lascivaroma_additional-texts/data/phi1351/phi005/phi1351.phi005.perseus-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng4.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi001/phi0448.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi002/phi0448.phi002.perseus-eng2.xml is 

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi002/phi0690.phi002.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi003/phi0690.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1017/phi011/phi1017.phi011.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0660/phi003/phi0660.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1002/phi001/phi1002.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0914/phi001/phi0914.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0917/phi001/phi0917.phi

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1056/phi001/phi1056.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi004/phi1351.phi004.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi002/phi1351.phi002.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0550/phi001/phi0550.phi001.perseus-eng1.xml is not present


# Génération des fichiers XML

## Nettoyage

In [3]:
!rm ./lemmatized/xml/*.xml
!rm ./lemmatized/*.xml

rm: cannot remove './lemmatized/*.xml': No such file or directory


## Récupération des données dépôts

In [4]:
REPOS = {
    
}
with open("../../data/raw/corpora.csv") as f:
    for lineno, line in enumerate(f):
        line = line.strip().split(";")
        if lineno == 0:
            headers = line
        else:
            line = dict(zip(headers, line))
            REPOS[line["Name"]] = line["Version"]

In [5]:
## Chargement des checksums
from pie_extended.utils.hashcheck import md5sum, check_checksum_from_file, read_checksum_csv
import os.path

former_checksums = read_checksum_csv(CHECKSUM_PATH)
xsl_checksum = md5sum(XSL)

repositories_map = {
    "OpenGreekAndLatin_Latin": "OpenGreekAndLatin/Latin",
    "OpenGreekAndLatin_csel-dev": "OpenGreekAndLatin/csel-dev",
    "lascivaroma_additional-texts": "lascivaroma/additional-texts",
    "lascivaroma_priapeia": "lascivaroma/priapeia",
    "PerseusDL_canonical-latinLit": "PerseusDL/canonical-latinLit",
    "ponteineptique_digiliblt": "ponteineptique/digiliblt"
}
file_to_urn = {
    plaintext.split("/")[-1][:-4]: {
        "plaintext_checksum": info.checksum,
        "source_checksum": info.source_checksum,
        "repo": repositories_map[
            info.source.replace("/home/thibault/dev/these/data/raw/corpora/", "").split("/")[0]
        ],
        "version": REPOS[
            repositories_map[
                info.source.replace("/home/thibault/dev/these/data/raw/corpora/", "").split("/")[0]
            ]
        ],
        
    }
    for plaintext, info in former_checksums.items()
}

## Génération

In [6]:
import regex as re
import os.path
import glob
import tqdm
import lxml.etree as etree



TEI = """<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader n="{tid}">
        <fileDesc>
            <titleStmt>
                <title>{title}</title>
                <author>{author}</author>
            </titleStmt>
            <publicationStmt>
                <publisher>
                    <persName>Thibault Clérice</persName>
                </publisher>
            </publicationStmt>
            <sourceDesc>
                <bibl>
                    <idno>{tid}</idno>
                    <link target="{uri}"/>
                    <dim source="xml" type="md5-checksum">{checksum_source}</dim>
                    <dim source="plaintext-transformation" type="md5-checksum">{checksum_text}</dim>
                    <dim source="xsl" type="md5-checksum">{checksum_xsl}</dim>
                </bibl>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
    <text n="{tid}">
        <body>
            
        </body>
    </text>
</TEI>
"""
reference = re.compile(r"(\[REF:[A-Za-z0-9\.]+\])")
for file in glob.glob("./lemmatized/tsv/*-pie.txt"):
    with open(file) as read:
        xml = file.replace("-pie.txt", ".xml")
        if xml == file:
            print(xml, file)
            break
        text_id = os.path.basename(file)[:-len("-pie.txt")]
        metadata = resolver.getMetadata(text_id)
        title = metadata.parent.get_label()
        author = metadata.parent.parent.get_label()
        version_info = file_to_urn[text_id]
        
        root = etree.fromstring(TEI.format(
                tid=text_id,
                title=title,
                author=author,
                checksum_source=version_info["source_checksum"],
                checksum_text=version_info["plaintext_checksum"],
                checksum_xsl=xsl_checksum,
                uri="https://github.com/{name}/archive/{version}.zip".format(
                    name=version_info["repo"], version=version_info["version"]
                )
            ))
        
        body = root.xpath("//t:body", namespaces={"t": "http://www.tei-c.org/ns/1.0"})[0]
        #with open(xml, "w") as write:
        last_seg = None
        for lineno, line in tqdm.tqdm(enumerate(read)):
            line = line.strip().split("\t")
            if lineno == 0:
                headers = line
                continue
            line = dict(zip(headers, line))


            # Deal with Segments
            if line["token"].startswith("[REF:"):
                section, *sid = line["token"][len("[REF:"):-1].split(".")
                sid = ".".join(sid)
                last_seg = etree.Element("{http://www.tei-c.org/ns/1.0}ab", type=section, n=f"{text_id}:{sid}")
                last_seg.tail = "\n            "
                last_seg.text = "\n                "
                body.append(last_seg)
                w_id = 1
                continue

            elif line["token"] in {"<", ">", "&"}:
                continue
            new_word = etree.Element(
                "{http://www.tei-c.org/ns/1.0}w",
                rend=section,
                n=sid,
                pos=line["pos"],
                msd=line["morph"],
                lemma=line["lemma"],
            )
            new_word.text = line["token"]
            new_word.tail = "\n                "
            last_seg.append(new_word)
            w_id += 1

        with open(xml, "w") as f:
            f.write(etree.tostring(root, encoding=str, pretty_print=True))
            
            

512it [00:00, 106691.36it/s]
5484it [00:00, 118361.59it/s]
469it [00:00, 45704.66it/s]
61977it [00:00, 116207.61it/s]
16190it [00:00, 81750.96it/s]
16489it [00:00, 105366.25it/s]
1646it [00:00, 47855.49it/s]
7975it [00:00, 120203.74it/s]
1081it [00:00, 58850.04it/s]
12251it [00:00, 116908.71it/s]
13292it [00:00, 108207.33it/s]
89337it [00:00, 118502.30it/s]
1478it [00:00, 13200.98it/s]
10259it [00:00, 122190.44it/s]
149112it [00:01, 121269.65it/s]
31696it [00:00, 74988.53it/s]
14201it [00:00, 93655.80it/s]
4281it [00:00, 81932.04it/s]
17780it [00:00, 115714.45it/s]
140802it [00:01, 121679.99it/s]
93136it [00:01, 70044.70it/s]
23227it [00:00, 80168.05it/s]
10698it [00:00, 94058.03it/s]
2663it [00:00, 79805.03it/s]
8112it [00:00, 119960.63it/s]
12836it [00:00, 117877.53it/s]
62208it [00:00, 115213.95it/s]
10147it [00:00, 63159.52it/s]
202it [00:00, 15654.15it/s]
679it [00:00, 102274.38it/s]
13018it [00:00, 110974.27it/s]
8117it [00:00, 94438.48it/s]
13145it [00:00, 110896.82it/s]
3778it 

4991it [00:00, 27790.12it/s]
26212it [00:00, 118481.86it/s]
118558it [00:00, 123882.09it/s]
736it [00:00, 4675.58it/s]
55090it [00:00, 126672.23it/s]
4911it [00:00, 49800.72it/s]
15690it [00:00, 119543.16it/s]
368it [00:00, 18258.32it/s]
43097it [00:00, 128079.68it/s]
18366it [00:00, 91056.68it/s]
4146it [00:00, 76849.85it/s]
17778it [00:00, 121442.95it/s]
25205it [00:00, 115680.91it/s]
87697it [00:00, 121105.72it/s]
7149it [00:00, 47116.79it/s]
6403it [00:00, 111087.28it/s]
5033it [00:00, 110283.01it/s]
6483it [00:00, 115983.66it/s]
4490it [00:00, 96161.32it/s]
18110it [00:00, 124582.54it/s]
4379it [00:00, 81033.71it/s]
3155it [00:00, 111691.87it/s]
4603it [00:00, 119147.25it/s]
16551it [00:00, 112107.94it/s]
10464it [00:00, 98561.85it/s]
28241it [00:00, 118663.51it/s]
8274it [00:00, 81173.43it/s]
16957it [00:00, 119472.90it/s]
5125it [00:00, 87111.15it/s]
1973it [00:00, 91592.27it/s]
32522it [00:00, 125106.53it/s]
18672it [00:00, 100612.08it/s]
8211it [00:00, 87610.96it/s]
114657it [

1878it [00:00, 110973.55it/s]
1863it [00:00, 110362.39it/s]
76520it [00:00, 120510.27it/s]
3348it [00:00, 27230.35it/s]
6535it [00:00, 115945.13it/s]
9317it [00:00, 115680.36it/s]
142975it [00:01, 119423.96it/s]
32846it [00:00, 76792.23it/s]
21813it [00:00, 104435.44it/s]
8521it [00:00, 88116.20it/s]
15463it [00:00, 112975.70it/s]
5502it [00:00, 75746.93it/s]
12786it [00:00, 107913.50it/s]
158049it [00:01, 120970.40it/s]
3354it [00:00, 15684.43it/s]
5053it [00:00, 98692.03it/s]
145249it [00:01, 117691.13it/s]
77931it [00:00, 95301.27it/s]
8044it [00:00, 47938.72it/s]
12009it [00:00, 88882.57it/s]
9397it [00:00, 75763.28it/s]
16364it [00:00, 104319.87it/s]
1509it [00:00, 45573.52it/s]
13353it [00:00, 116119.53it/s]
2677it [00:00, 71562.93it/s]
5886it [00:00, 108187.22it/s]
1852it [00:00, 68340.47it/s]
5189it [00:00, 95020.86it/s]
2884it [00:00, 72999.00it/s]
44631it [00:00, 117665.39it/s]
30267it [00:00, 97764.75it/s]
21472it [00:00, 99590.73it/s]
24062it [00:00, 110394.17it/s]
105342it

## Déplacement des nouveaux fichiers.

In [7]:
!mkdir -p lemmatized/xml
!mv lemmatized/tsv/*.xml lemmatized/xml/

# Zip pour partage des données

In [8]:
!zip data lemmatized/xml/*.xml

  adding: lemmatized/xml/urn:cts:greekLit:tlg0526.tlg003.opp-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:anthoLat.anthoLat.musisquedeoque-lat1.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:dlt000006.001.csl-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0022.phi001.thayer-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi001.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi002.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi003.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi004.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi005.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi006.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi007.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/

  adding: lemmatized/xml/urn:cts:latinLit:phi0620.phi001.perseus-lat3.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi001.perseus-lat3.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi001.perseus-lat4.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi002.perseus-lat3.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi002.perseus-lat4.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi003.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi007.thayer-lat1.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi008.thayer-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0660.phi001.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0660.phi003.perseus-lat2.xml (deflated 88%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0684.phi001.remacle-lat1.xml (deflated 92%)
  adding: lemmatized/xm

  adding: lemmatized/xml/urn:cts:latinLit:phi1348.abo022.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi1351.phi001.perseus-lat1.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi1351.phi002.perseus-lat1.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:phi1351.phi004.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi1351.phi005.lascivaroma-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi1672.phi002.digilibLT-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2003.phi001.opp-lat3.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2003.phi002.opp-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2003.phi003.opp-lat2.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2003.phi004.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi001.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:ct

  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa036.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa037.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa037a.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa037c.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa038a.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa040.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa041.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa042.opp-lat1.xml (deflated 94%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa042a.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa044.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa046.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinL

  adding: lemmatized/xml/urn:cts:latinLit:stoa0089.stoa009.perseus-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0089.stoa010.perseus-lat2.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0089.stoa011.perseus-lat2.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0089.stoa012.perseus-lat2.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0089.stoa013.perseus-lat2.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0089.stoa014.perseus-lat2.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0089a.stoa001.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0089a.stoa002.opp-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0096.stoa003.opp-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0096.stoa004.opp-lat2.xml (deflated 90%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa001.opp-lat1.xml (deflated 91%)
  adding: lemmat

  adding: lemmatized/xml/urn:cts:latinLit:stoa0255.stoa010.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0255.stoa011.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0255.stoa012.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0255.stoa013.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0255.stoa014.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0270.stoa001.opp-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0270.stoa002.opp-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0270.stoa003.opp-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0270.stoa005.opp-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0270p.stoa001.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa001.opp-lat1.xml (deflated 91%)
  adding: lemmatized/x